**FAKE NEWS CLASSIFIER USING LSTM**

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('train.csv')

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
## Drop non values
df = df.dropna()

In [7]:
x = df.drop('label', axis = 1)

In [8]:
y = df['label']

In [9]:
x.shape

(18285, 4)

In [10]:
y.shape

(18285,)

In [11]:
import tensorflow as tf

In [12]:
tf.__version__

'2.16.1'

In [53]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences  ## pad_sequences is to fix the input length in embedding layer, to make sentences equal in lengh in pre and post.
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [16]:
## vocabulary size
voc_size = 5000

**ONE HOT REPRESENTATION**

In [17]:
messages = x.copy()

In [18]:
messages.reset_index(inplace = True)

In [19]:
import nltk
import re
from nltk.corpus import stopwords

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Karthik M
[nltk_data]     hadagali\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
## Dataset Preprocessing 
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    # substituiting all characters above a-zA_Z
    review = re.sub('[^a-zA-Z]', ' ' , messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [25]:
onehot_repr = [one_hot(words,voc_size)for words in corpus]
onehot_repr

[[1519, 3987, 123, 2155, 3610, 1147, 3232, 1012, 3569, 1243],
 [3156, 945, 2171, 699, 3758, 1178, 4085],
 [2469, 3356, 4923, 4939],
 [883, 3379, 3139, 478, 1822, 487],
 [4282, 3758, 539, 2419, 4409, 2355, 3758, 4602, 4324, 2497],
 [2836,
  2695,
  1611,
  4894,
  2021,
  2895,
  1457,
  643,
  1378,
  4846,
  3434,
  3372,
  36,
  2006,
  4085],
 [1738, 2253, 2692, 238, 2036, 3160, 2352, 2258, 2893, 2864, 3126],
 [3568, 4934, 3650, 669, 4189, 2901, 2895, 2556, 2893, 2864, 3126],
 [1172, 3765, 293, 530, 2388, 3776, 3411, 4986, 2895, 3667],
 [696, 3833, 2540, 416, 3089, 2584, 3837, 377],
 [406, 2632, 4352, 594, 4013, 3439, 3767, 1803, 2780, 3410, 3783],
 [478, 2328, 3610, 3776, 2895, 4189],
 [1582, 1048, 3892, 1680, 3890, 3173, 3733, 2470, 1659],
 [2377, 701, 3072, 1188, 638, 2859, 381, 2893, 2864, 3126],
 [3945, 4223, 1101, 2167, 349, 2893, 2864, 3126],
 [3665, 3910, 2609, 4478, 703, 3749, 1599, 3690, 3759, 2264],
 [2917, 3070, 945],
 [3152, 1634, 3536, 2537, 2895, 4700, 4865, 4085],
 [

**EMBEDDING REPRESENTATION**

In [27]:
## to fix all the sentence lenth into same lenth we use pad sequences
sent_length = 20 
embedded_docs = pad_sequences(onehot_repr, padding = 'pre', maxlen = sent_length)
print(embedded_docs)

[[   0    0    0 ... 1012 3569 1243]
 [   0    0    0 ... 3758 1178 4085]
 [   0    0    0 ... 3356 4923 4939]
 ...
 [   0    0    0 ... 2893 2864 3126]
 [   0    0    0 ... 1303 1827 3980]
 [   0    0    0 ... 1711 1095 2397]]


In [28]:
len(embedded_docs)

18285

In [34]:
embedded_docs[18282]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4808,
       1249, 2953, 2303, 1761,  961, 4858, 2893, 2864, 3126])

In [37]:
## creating model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [38]:
len(embedded_docs),y.shape

(18285, (18285,))

In [42]:
import numpy as np
x_final = np.array(embedded_docs)
y_final = np.array(y)

In [43]:
x_final.shape, y_final.shape

((18285, 20), (18285,))

In [44]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size = 0.33, random_state = 42)

**MODEL TRAINING**

In [45]:
model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 15s 61ms/step - accuracy: 0.8013 - loss: 0.4204 - val_accuracy: 0.8936 - val_loss: 0.2257
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 13s 66ms/step - accuracy: 0.9395 - loss: 0.1504 - val_accuracy: 0.9153 - val_loss: 0.2029
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 13s 68ms/step - accuracy: 0.9644 - loss: 0.0952 - val_accuracy: 0.9092 - val_loss: 0.2265
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 59ms/step - accuracy: 0.9736 - loss: 0.0753 - val_accuracy: 0.9077 - val_loss: 0.2994
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 49ms/step - accuracy: 0.9820 - loss: 0.0481 - val_accuracy: 0.9077 - val_loss: 0.3334
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 50ms/step - accuracy: 0.9881 - loss: 0.0339 - val_accuracy: 0.9039 - val_loss: 0.3262
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 56ms/step - accuracy: 0.9948 - loss: 0.0209 - val_accuracy: 0.8993 - val_loss: 0.3799
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - accuracy: 0.9972 - loss: 0.0140 - v

**ADDING DROPOUT**

In [57]:
from tensorflow.keras.layers import Dropout
## creating model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size , embedding_vector_features, input_length = sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

D:\Anaconda_24\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [58]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [59]:
import numpy as np
x_final = np.array(embedded_docs)
y_final = np.array(y)

In [60]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size = 0.33, random_state = 42)

In [61]:
model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 14s 41ms/step - accuracy: 0.7869 - loss: 0.4528 - val_accuracy: 0.9104 - val_loss: 0.2085
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - accuracy: 0.9357 - loss: 0.1617 - val_accuracy: 0.9180 - val_loss: 0.2016
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.9582 - loss: 0.1149 - val_accuracy: 0.9142 - val_loss: 0.2132
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - accuracy: 0.9722 - loss: 0.0818 - val_accuracy: 0.9117 - val_loss: 0.2389
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - accuracy: 0.9768 - loss: 0.0627 - val_accuracy: 0.9104 - val_loss: 0.2816
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 34ms/step - accuracy: 0.9848 - loss: 0.0485 - val_accuracy: 0.9029 - val_loss: 0.3009
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 34ms/step - accuracy: 0.9890 - loss: 0.0339 - val_accuracy: 0.9075 - val_loss: 0.3265
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 39ms/step - accuracy: 0.9919 - loss: 0.0276 - val_acc

**Performance Metrics and Accuracy**

In [63]:
y_pred = model.predict_classes(x_test)

AttributeError: 'Sequential' object has no attribute 'predict_classes'

In [48]:
from sklearn.metrics import confusion_matrix

In [50]:
confusion_matrix(y_test, y_pred)

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [54]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

ValueError: Classification metrics can't handle a mix of binary and continuous targets